# Assignment 5 언어처리기반 추천시스템

In [161]:
import pandas as pd
import urllib.request
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [162]:
# CSV 파일 경로
file_path = '/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/University/Ulsan university/Electrical & Electronic Engineering/4학년/2학기/Ai추천시스템/Textbook-code&data-set/data2/movies_metadata.csv'

try:
    movies = pd.read_csv(file_path, encoding='utf-8')  # 기본 UTF-8
except UnicodeDecodeError:
    # UTF-8 디코딩 실패 시 다른 인코딩 사용
    movies = pd.read_csv(file_path, encoding='ISO-8859-1')

# 데이터 확인
movies.head()

/var/folders/g6/3qldr4jd21lbltbbdswtbqkm0000gn/T/ipykernel_78928/2342080988.py:8: DtypeWarning: Columns (2,36,37,40) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(file_path, encoding='ISO-8859-1')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FALSE,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FALSE,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44'],
      dtype='object')

In [164]:
text = movies[['overview']]
text.head()

,overview
0,"Led by Woody, Andy's toys live happily in his ..."
1,When siblings Judy and Peter discover an encha...
2,A family wedding reignites the ancient feud be...
3,"Cheated on, mistreated and stepped on, the wom..."
4,Just when George Banks has recovered from his ...


In [165]:
text.isna().sum()

overview    953
dtype: int64

In [166]:
# 결측값 제거
text = text.dropna(subset=['overview'])

## 텍스트 전처리

In [167]:
# NLTK로부터 불용어를 받아온다.
nltk.download('stopwords')
stop_words = stopwords.words('english')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /Users/jun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/jun/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [168]:
text.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44489 entries, 0 to 45441
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   overview  44489 non-null  object
dtypes: object(1)
memory usage: 695.1+ KB


In [169]:
text.isnull().sum()

overview    0
dtype: int64

In [170]:
# 결측값 제거
text = text.dropna(subset=['overview'])

In [171]:
text['overview'] = text.apply(lambda row: nltk.word_tokenize(row['overview']), axis=1)

In [172]:
text.head()

,overview
0,"[Led, by, Woody, ,, Andy, 's, toys, live, happ..."
1,"[When, siblings, Judy, and, Peter, discover, a..."
2,"[A, family, wedding, reignites, the, ancient, ..."
3,"[Cheated, on, ,, mistreated, and, stepped, on,..."
4,"[Just, when, George, Banks, has, recovered, fr..."


In [173]:
stop_words = stopwords.words('english')
text['overview'] = text['overview'].apply(lambda x: [word for word in x if word not in (stop_words)])

In [174]:
text.head()

,overview
0,"[Led, Woody, ,, Andy, 's, toys, live, happily,..."
1,"[When, siblings, Judy, Peter, discover, enchan..."
2,"[A, family, wedding, reignites, ancient, feud,..."
3,"[Cheated, ,, mistreated, stepped, ,, women, ho..."
4,"[Just, George, Banks, recovered, daughter, 's,..."


In [175]:
# 3인칭 단수 표현을 1인칭으로 바꾸고, 과거 현재형 동사를 현재형으로 q변경
nltk.download('wordnet')
text['overview'] = text['overview'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
print(text.head(5))

[nltk_data] Downloading package wordnet to /Users/jun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                            overview
0  [Led, Woody, ,, Andy, 's, toy, live, happily, ...
1  [When, siblings, Judy, Peter, discover, enchan...
2  [A, family, wed, reignite, ancient, feud, next...
3  [Cheated, ,, mistreat, step, ,, women, hold, b...
4  [Just, George, Banks, recover, daughter, 's, w...


In [176]:
#길이가 3이하인 단어에 대해서 제거
tokenized_doc = text['overview'].apply(lambda x: [word for word in x if len(word) > 3])
print(tokenized_doc[:5])

0    [Woody, Andy, live, happily, room, Andy, birth...
1    [When, siblings, Judy, Peter, discover, enchan...
2    [family, reignite, ancient, feud, next-door, n...
3    [Cheated, mistreat, step, women, hold, breath,...
4    [Just, George, Banks, recover, daughter, recei...
Name: overview, dtype: object


In [177]:
print(len(tokenized_doc))
print(len(text))

44489
44489


In [178]:
tokenized_doc.isnull().sum()

0

In [179]:
text.isna().sum()

overview    0
dtype: int64

## TF-IDF 행렬 만들기

In [180]:
# tokenized_doc와 text 크기 확인
if len(tokenized_doc) != len(text):
    raise ValueError("tokenized_doc와 text의 크기가 일치하지 않습니다.")

# 결측값 처리 및 데이터 정리
tokenized_doc = [x if isinstance(x, list) else [] for x in tokenized_doc]

# 역토큰화 수행
detokenized_doc = []
for i in range(len(tokenized_doc)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

# 결과를 다시 text 데이터프레임에 저장
text['overview'] = detokenized_doc

In [181]:
text.head()

,overview
0,Woody Andy live happily room Andy birthday bri...
1,When siblings Judy Peter discover enchant boar...
2,family reignite ancient feud next-door neighbo...
3,Cheated mistreat step women hold breath wait e...
4,Just George Banks recover daughter receive new...


In [182]:
# 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외
# bigram도 포함
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
feat_vect = count_vectorizer.fit_transform(text['overview'])

X = feat_vect  # X를 문서-단어 행렬로 정의
print('CountVectorizer Shape:', X.shape)

CountVectorizer Shape: (44489, 1000)


## 토픽 모델링

In [183]:
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1)
lda_top = lda_model.fit_transform(feat_vect)
print(lda_model.components_)
print(lda_model.components_.shape)

[[1.00022783e-01 1.00049590e-01 1.00032645e-01 ... 1.00024575e-01
  1.00036362e-01 1.00029831e-01]
 [8.25502639e+01 2.57776880e+02 1.00033165e-01 ... 8.08871289e-01
  1.65616927e-01 3.54189840e+01]
 [1.00028377e-01 1.00031125e-01 1.87103296e+02 ... 1.00016587e-01
  1.00022298e-01 7.37269080e+01]
 ...
 [1.12070391e-01 1.00023410e-01 1.53259953e+02 ... 1.00011890e-01
  1.00042890e-01 1.00060830e-01]
 [1.00789694e-01 1.00040458e-01 1.00023995e-01 ... 1.00022364e-01
  1.32808614e+02 1.00114212e-01]
 [2.01624680e+02 1.00389326e-01 1.00032281e-01 ... 1.00014113e-01
  2.74027489e+01 1.19127770e+02]]
(10, 1000)


In [184]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = count_vectorizer.get_feature_names_out()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: [('family', 1851.46), ('father', 1534.81), ('fight', 1098.26), ('brother', 1061.37), ('something', 859.88)]
Topic 2: [('home', 1781.87), ('begin', 1371.32), ('murder', 1365.57), ('become', 1338.14), ('turn', 1325.18)]
Topic 3: [('house', 1878.74), ('police', 1607.86), ('however', 1364.57), ('money', 1055.99), ('help', 973.19)]
Topic 4: [('world', 2313.78), ('beautiful', 1270.64), ('change', 1245.33), ('story', 1194.64), ('village', 1138.06)]
Topic 5: [('year', 2826.85), ('girl', 2668.69), ('night', 1749.22), ('mother', 1715.15), ('city', 1617.26)]
Topic 6: [('woman', 2026.35), ('include', 1267.1), ('call', 1206.43), ('music', 1080.64), ('death', 950.13)]
Topic 7: [('fall', 2076.75), ('marry', 1464.48), ('meet', 1421.95), ('years', 1416.48), ('show', 1397.93)]
Topic 8: [('school', 2523.94), ('high', 1774.16), ('best', 1515.93), ('story', 1191.86), ('special', 1138.19)]
Topic 9: [('town', 2091.84), ('years', 1466.03), ('small', 1328.11), ('make', 1219.42), ('later', 1127.9)]
Top

In [ ]:
import pyLDAvis

pyLDAvis.enable_notebook()

doc_lengths = feat_vect.sum(axis=1).A1
term_frequency = X.sum(axis=0).getA1() 

doc_topic_dist = lda_model.transform(feat_vect)

vis = pyLDAvis.prepare(lda_model.components_, doc_topic_dist, doc_lengths=doc_lengths, vocab=terms, term_frequency=term_frequency)
pyLDAvis.display(vis)


## 문서-토픽 분포를 바탕으로 코사인 유사도 계산

In [ ]:
cosine_sim = cosine_similarity(lda_top, lda_top)

/opt/anaconda3/envs/ai_recmd/lib/python3.11/site-packages/sklearn/utils/extmath.py:205: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [191]:
cosine_sim

array([[1.        , 0.02008159, 0.01527283, ..., 0.03764723, 0.03293648,
        0.03539051],
       [0.02008159, 1.        , 0.09174199, ..., 0.08054935, 0.20340678,
        0.04005678],
       [0.01527283, 0.09174199, 1.        , ..., 0.26340827, 0.43876228,
        0.02987353],
       ...,
       [0.03764723, 0.08054935, 0.26340827, ..., 1.        , 0.43953151,
        0.06842892],
       [0.03293648, 0.20340678, 0.43876228, ..., 0.43953151, 1.        ,
        0.72627126],
       [0.03539051, 0.04005678, 0.02987353, ..., 0.06842892, 0.72627126,
        1.        ]])

## NumPy 코사인 유사도 배열을 기반으로 특정 영화와 유사한 영화를 추천
- 유사도 계산 후 DataFrame 변환하는 작업 중 지속적인 커널 충돌 발생으로 NumPy 코사인 유사도 기반 추천 구현

In [ ]:
def lda_recommender(title, n_of_recomm=10):
    
    # 입력된 영화 제목으로 인덱스를 찾기
    idx = movies[movies['title'] == title].index[0]

    sim_scores = list(enumerate(cosine_sim[idx])) # 배열에서 기준 영화와 다른 모든 영화 간의 유사도를 가져옴
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n_of_recomm+1] # 유사도 순으로 정렬 (자기자신 제외)

    # 추천 영화의 인덱스를 가져와 제목 출력
    movie_indices = [i[0] for i in sim_scores] # 상위 추천 영화들의 인덱스만 추출
    return movies['title'].iloc[movie_indices] # 추천 영화들의 제목 리스트 반환

In [188]:
movies['title']

0                          Toy Story
1                            Jumanji
2                   Grumpier Old Men
3                  Waiting to Exhale
4        Father of the Bride Part II
                    ...             
45437                         Subdue
45438            Century of Birthing
45439                       Betrayal
45440               Satan Triumphant
45441                       Queerama
Name: title, Length: 45442, dtype: object

In [192]:
# 'Toy Story'와 유사한 영화 추천
recommended_movies = lda_recommender('Toy Story')
print(recommended_movies)

39                                Richard III
100                             Unforgettable
7                                Sudden Death
1496                                    Mondo
41618     Froning: The Fittest Man In History
33881               Summer Time Machine Blues
37532    La felicitÃ  Ã¨ un sistema complesso
11139                                   Crank
7751                                Possessed
18646                          A Cat in Paris
Name: title, dtype: object


In [193]:
recommended_movies = lda_recommender('Father of the Bride Part II')
print(recommended_movies)

10       Dracula: Dead and Loving It
34                  Dead Man Walking
39520                     Homme Less
125                   Jupiter's Wife
104           Muppet Treasure Island
34517                The Hot Flashes
1383                   McHale's Navy
18031                      Anonymous
2817                       Indochine
20070            Stand Up and Cheer!
Name: title, dtype: object
